In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame({
    "가로":[10, 20, 30],
    "세로":[5, 10, 15]
})
df

,가로,세로
0,10,5
1,20,10
2,30,15


In [3]:
df["면적"] = df.eval("가로*세로")
df

,가로,세로,면적
0,10,5,50
1,20,10,200
2,30,15,450


In [4]:
target = {"가로": 10, "세로": 20, "면적": 200, "최장길이": 20, "최소길이": 10}

In [5]:
import functools # not required, but helps in production
def unpack_df_columns(func):
    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):
        series = args[0]
        return func(*series.values)
    return _unpack_df_columns

In [6]:
@unpack_df_columns
def 길이리스트만들기(길이1, 길이2):
    result = []
    가로 = max(길이1, 길이2)
    세로 = min(길이1, 길이2)
    result.append(가로)
    result.append(가로)
    result.append(세로)
    result.append(세로)
    return result

In [7]:
df["길이리스트"] = df[["가로", "세로"]].apply(길이리스트만들기, axis=1)
df

,가로,세로,면적,길이리스트
0,10,5,50,"[10, 10, 5, 5]"
1,20,10,200,"[20, 20, 10, 10]"
2,30,15,450,"[30, 30, 15, 15]"


In [8]:
df["최장길이"] = df["길이리스트"].apply(lambda x : max(x))
df["최소길이"] = df["길이리스트"].apply(lambda x : min(x))
df

,가로,세로,면적,길이리스트,최장길이,최소길이
0,10,5,50,"[10, 10, 5, 5]",10,5
1,20,10,200,"[20, 20, 10, 10]",20,10
2,30,15,450,"[30, 30, 15, 15]",30,15


In [9]:
@unpack_df_columns
def 면적_최장길이적합도_검토(정반면적, 정반최장길이):
    global target
    
    if 정반면적 >= target["면적"] and 정반최장길이 >= target["최장길이"]:
        return "적합"
    else:
        return "부적합"

In [10]:
df["적합도검토"] = df[["면적", "최장길이"]].apply(면적_최장길이적합도_검토, axis=1)
df

,가로,세로,면적,길이리스트,최장길이,최소길이,적합도검토
0,10,5,50,"[10, 10, 5, 5]",10,5,부적합
1,20,10,200,"[20, 20, 10, 10]",20,10,적합
2,30,15,450,"[30, 30, 15, 15]",30,15,적합


In [11]:
# 만약 target을 2번 정반에 배치했다면...잔여 정반 사이지를 어떻게 표현할 것인가?

In [12]:
# target = {"가로": 30, "세로": 10, "면적": 300, "최장길이": 30, "최소길이": 10}
target = {"가로": 20, "세로": 5, "면적": 100, "최장길이": 20, "최소길이": 5}
길이리스트 = [30, 30, 15, 15]

In [13]:
if min(길이리스트) == target["최소길이"] and max(길이리스트) >= target["최장길이"]:
    길이리스트[0] = 길이리스트[0] - target["최장길이"]
    길이리스트[1] = 길이리스트[1] - target["최장길이"]
    길이리스트[2] = target["최소길이"]
    길이리스트[3] = target["최소길이"]

if max(길이리스트) == target["최장길이"] and min(길이리스트) >= target["최소길이"]:
    길이리스트[0] = target["최장길이"]
    길이리스트[1] = target["최장길이"]
    길이리스트[2] = 길이리스트[2] - target["최소길이"]
    길이리스트[3] = 길이리스트[3] - target["최소길이"]

else:
    길이리스트[0] = 길이리스트[0] - target["최장길이"]
    길이리스트[2] = 길이리스트[2] - target["최소길이"]

In [25]:
def 배치후잔여정반길이리스트(정반길이리스트, 블록최장길이, 블록최소길이):
    
    if min(정반길이리스트) == 블록최소길이 and max(정반길이리스트) >= 블록최장길이:
        정반길이리스트[0] = 정반길이리스트[0] - 블록최장길이
        정반길이리스트[1] = 정반길이리스트[1] - 블록최장길이
        정반길이리스트[2] = 블록최소길이
        정반길이리스트[3] = 블록최소길이

    elif max(정반길이리스트) == 블록최장길이 and min(정반길이리스트) >= 블록최소길이:
        정반길이리스트[0] = 블록최장길이
        정반길이리스트[1] = 블록최장길이
        정반길이리스트[2] = 정반길이리스트[2] - 블록최소길이
        정반길이리스트[3] = 정반길이리스트[3] - 블록최소길이

    else:
        정반길이리스트[0] = 정반길이리스트[0] - 블록최장길이
        정반길이리스트[2] = 정반길이리스트[2] - 블록최소길이
    
    return 정반길이리스트
    

In [26]:
정반길이리스트 = [20, 20, 10, 10]
블록최장길이 = 10
블록최소길이 = 10
정반길이리스트 = 배치후잔여정반길이리스트(정반길이리스트, 블록최장길이, 블록최소길이)
정반길이리스트

[10, 10, 10, 10]

In [16]:
# 잔여면적 = []
# if 길이리스트[0] == 길이리스트[1]:
#     잔여면적.append(길이리스트[0]*길이리스트[3])
# else:
#     잔여면적  = []
#     잔여면적.append(길이리스트[0]*길이리스트[3])
#     잔여면적.append(길이리스트[1]*길이리스트[2])
# 잔여면적

In [17]:
def 면적리스트구하기(길이리스트):
    면적리스트 = []
    if 길이리스트[0] == 길이리스트[1]:
        면적리스트.append(길이리스트[0]*길이리스트[3])
    else:
        면적리스트  = []
        면적리스트.append(길이리스트[0]*길이리스트[3])
        면적리스트.append(길이리스트[1]*길이리스트[2])
    return 면적리스트

In [18]:
면적리스트구하기(길이리스트1)

[150, 210]

In [19]:
## 자식정반에 면적이 2개 이상인 경우는???
def 면적_최장길이적합도_검토1(면적리스트, 정반길이리스트, 블록면적, 블록최장길이):
    result = []
    for i in range(len(면적리스트)):
        if i == 0:
            if 면적리스트[i] >= 블록면적 and max(정반길이리스트[0], 정반길이리스트[3]) >= 블록최장길이:
                result.append("적합")
            else:
                result.append("부적합")
        
        else:
            if 면적리스트[i] >= 블록면적 and max(정반길이리스트[1], 정반길이리스트[2]) >= 블록최장길이:
                result.append("적합")
            else:
                result.append("부적합")
                
    return result

In [20]:
면적리스트 = [150, 210]
정반길이리스트 = [10, 30, 7, 15]
블록면적 = 40
블록최장길이 = 20
면적_최장길이적합도_검토1(면적리스트, 정반길이리스트, 블록면적, 블록최장길이)

['부적합', '적합']

In [22]:
면적리스트 = [100]
정반길이리스트 = [10, 10, 10, 10]
블록면적 = 120
블록최장길이 = 12
면적_최장길이적합도_검토1(면적리스트, 정반길이리스트, 블록면적, 블록최장길이)

['부적합']